In [1]:
########################################################################
## Author: Shane Miller (620042891)
## Sources: 
## https://thinkingneuron.com/predicting-stock-prices-using-deep-learning-lstm-model-in-python/
## https://www.relataly.com/stock-market-prediction-using-multivariate-time-series-in-python/1815/#h-summary
########################################################################

## Mount Google Drive 

In [2]:
currentWorkingDir = !pwd
defaultWorkingDir = "/content"

if ( currentWorkingDir[0] == defaultWorkingDir ):
  from google.colab import drive

  drive.mount('/content/drive')
      
  %cd "/content/drive/My Drive/Colab Notebooks/stock_portfolio"
else:
  print("Currenting running app from: ")
  !pwd

Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/stock_portfolio


In [3]:
import math # Mathematical functions 
import numpy as np # Fundamental package for scientific computing with Python
import pandas as pd # Additional functions for analysing and manipulating data
import time
import os
import os.path
from datetime import date, timedelta, datetime # Date Functions
from pandas.plotting import register_matplotlib_converters # This function adds plotting functions for calender dates
import matplotlib.pyplot as plt # Important package for visualization - we use this to plot the market data
import matplotlib.dates as mdates # Formatting dates
import tensorflow as tf
from tensorflow import keras



from sklearn.metrics import mean_absolute_error, mean_squared_error # Packages for measuring model performance / errors
from tensorflow.keras.models import Sequential # Deep learning library, used for neural networks
from tensorflow.keras.layers import LSTM, Dense, Dropout # Deep learning classes for recurrent and regular densely-connected layers
from tensorflow.keras.callbacks import EarlyStopping # EarlyStopping during model training
from sklearn.preprocessing import RobustScaler, MinMaxScaler # This Scaler removes the median and scales the data according to the quantile range to normalize the price data 
#import seaborn as sns
 
# To remove the scientific notation from numpy arrays
np.set_printoptions(suppress=True)

# Define Functions

In [4]:
def inScopeStocks():
  ### Function to determine stocks in scope for analysis/prediction
  # Fetching the data
  raw_df              = pd.read_csv(  'jse_main_price_2016-2020.csv',
                                      infer_datetime_format=True, 
                                      sep=','
                      )
  
  # Group stocks to determine how often they were traded during the year
  StockData_bySymbol  = raw_df[['Symbol','Last_Traded']] \
                          .groupby(['Symbol']) \
                          .count() \
                          .sort_values(by=['Last_Traded'],ascending=False)
  
  # Determine and filter for stocks that traded for at least 70% of available
  # trading days/period
  working_days_year   = 252       # Working days for 2020 used 
  no_years            = 5         # 2016-2020
  filter              = 0.70      # Based on scope outlined in proposal
  minDaysStockTraded  = (working_days_year * no_years) * filter
  
  # Reset the index to ensure that 'Symbol' is a column
  StockData_bySymbol  = StockData_bySymbol.reset_index()
  
  # Create a list of stocks that meet the criteria
  criteria_forInScope = StockData_bySymbol['Last_Traded'] >= minDaysStockTraded
  inScopeStocks       = StockData_bySymbol.loc[criteria_forInScope,"Symbol"]

  return   inScopeStocks

In [5]:
def create_LSTM_mv_model(StockData, symbol_str, features, epochs):
    
  def LSTM_mv_scaleData(StockData):
                          
      # Replace nan with zero
      df_m = StockData.fillna(0)

      # create index with date
      df_m = df_m.set_index("Date")

      # Indexing Batches
      train_df = df_m.sort_values(by=['Date']).copy()

      # We save a copy of the dates index, before we reset it to numbers
      date_index = train_df.index

      # We reset the index, so we can convert the date-index to a number-index
      train_df = train_df.reset_index(drop=True).copy()
    
      # Create the dataset with features and filter the data to the list of FEATURES
      data = pd.DataFrame(train_df)
      data_filtered = data[features]

      # We add a prediction column and set dummy values to prepare the data for scaling
      data_filtered_ext = data_filtered.copy()
      data_filtered_ext['Prediction'] = data_filtered_ext['Close_Price']
      
      # Get the number of rows in the data
      nrows = data_filtered.shape[0]
      print(nrows)

      # Convert the data to numpy values
      np_data_unscaled = np.array(data_filtered)
      print(data_filtered.tail(5))
      print(np_data_unscaled[0])
      np_data = np.reshape(np_data_unscaled, (nrows, -1))
      print(np_data.shape)

      # Transform the data by scaling each feature to a range between 0 and 1
      scaler = MinMaxScaler()
      np_data_scaled = scaler.fit_transform(np_data_unscaled)

      # Creating a separate scaler that works on a single column for scaling predictions
      scaler_pred = MinMaxScaler()
      df_Close = pd.DataFrame(data_filtered_ext['Close_Price'])
      np_Close_scaled = scaler_pred.fit_transform(df_Close)
      
      
      return (np_data_scaled,
              data_filtered,
              data.columns.get_loc("Close_Price"),
              scaler_pred,
              date_index
      )

  np_data_scaled, \
  data_filtered,  \
  index_Close,    \
  scaler_pred,    \
  date_index      = LSTM_mv_scaleData(StockData)
    
  def LSTM_mv_DataPreparation (sequence_length):

      # sequence_length: Next few day's Price Prediction is based on the no of previous day's prices
      #            that is set here.

      # Split the training data into train and train data sets
      # As a first step, we get the number of rows to train the model on 80% of the data 
      train_data_len = math.ceil(np_data_scaled.shape[0] * 0.8)
      print(train_data_len)

      # Create the training and test data
      train_data = np_data_scaled[0:train_data_len, :]
      print(train_data[0][0])
      test_data  = np_data_scaled[train_data_len - sequence_length:, :]

      # The RNN needs data with the format of [samples, time steps, features]
      # Here, we create N samples, sequence_length time steps per sample, and the no of features
      def partition_dataset(sequence_length, data):
          x, y = [], []
          data_len = data.shape[0]
          for i in range(sequence_length, data_len):
              x.append(data[i-sequence_length:i,:]) #contains sequence_length values 0-sequence_length * columns
              y.append(data[i, index_Close]) #contains the prediction values for validation
          
          # Convert the x and y to numpy arrays
          x = np.array(x)
          y = np.array(y)
          return x, y

      # Generate training data and test data
      x_train, y_train = partition_dataset(sequence_length, train_data)
      x_test, y_test   = partition_dataset(sequence_length, test_data)    
      
      print(index_Close)

      # Print the shapes: the result is: (rows, training_sequence, features) (prediction value, )
      print(x_train.shape, y_train.shape)
      print(x_test.shape, y_test.shape)

      # Validate that the prediction value and the input match up
      # The last close price of the second input sample should equal the first prediction value
      print(x_train[1][sequence_length-1][index_Close])
      print(y_train[0])


      return ( x_train, x_test,
                y_train, y_test
      )   
    
  x_train, x_test, \
  y_train, y_test = LSTM_mv_DataPreparation(  sequence_length = 50 )

  def LSTM_mv_MultiModel( epochs ):

      # Configure the neural network model
      model = Sequential()

      # Model with n_neurons = inputshape Timestamps, each with x_train.shape[2] variables
      n_neurons = x_train.shape[1] * x_train.shape[2]
      print(n_neurons, x_train.shape[1], x_train.shape[2])
      model.add(LSTM( n_neurons, 
                      return_sequences=True, 
                      input_shape=(x_train.shape[1], x_train.shape[2])
                  )
              )    
      model.add(LSTM( n_neurons, 
                      return_sequences=False)
              )
      
      model.add(Dense(5))     
      model.add(Dense(1))

      # Compile the model
      model.compile(optimizer='adam', loss='mse')

        
      ##### Measuring the time taken by the model to train
      StartTime           = time.time()
      
      early_stop = EarlyStopping(monitor='loss', patience=5, verbose=1)
      
      # Fitting the RNN to the Training set
      model.fit(      x_train, 
                      y_train, 
                      batch_size = 16, 
                      epochs = epochs,
                      validation_data=(x_test, y_test)
      )
      
      EndTime             = time.time()
      #####
      
      
      # Return the model and the total time taken to run the model
      return ( model, round((EndTime-StartTime)/60)) 


  # Run the model
  lstm_model, mDur    = LSTM_mv_MultiModel( epochs = epochs )

  def LSTM_mv_ModelEvaluation():
  
      # Get the predicted values
      y_pred_scaled = lstm_model.predict(x_test)

      # Unscale the predicted values
      y_pred = scaler_pred.inverse_transform(y_pred_scaled)
      y_test_unscaled = scaler_pred.inverse_transform(y_test.reshape(-1, 1))

      # Mean Absolute Error (MAE)
      MAE = mean_absolute_error(y_test_unscaled, y_pred)
      print(f'Median Absolute Error (MAE): {np.round(MAE, 2)}')

      # Median Absolute Error (MedAE)
      MEDAE = np.median(abs(y_pred - y_test_unscaled))
      print(f'Median Absolute Error (MedAE): {np.round(MEDAE, 2)}')

      # Mean Squared Error (MSE)
      MSE = np.square(np.subtract(y_pred, y_test_unscaled)).mean()
      print('Mean Squared Error (MSE): ' + str(np.round(MSE, 2)))

      # Root Mean Squarred Error (RMSE) 
      RMSE = np.sqrt(np.mean(np.square(y_pred - y_test_unscaled)))
      print('Root Mean Squared Error (RMSE): ' + str(np.round(RMSE, 2)))

      # Mean Absolute Percentage Error (MAPE)
      MAPE = np.mean((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
      print(f'Mean Absolute Percentage Error (MAPE): {np.round(MAPE, 2)} %')

      # Median Absolute Percentage Error (MDAPE)
      MDAPE = np.median((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled)) ) * 100
      print(f'Median Absolute Percentage Error (MDAPE): {np.round(MDAPE, 2)} %')
  
      return (MAE, MEDAE, MSE, RMSE, MAPE, MDAPE)
    
  MAE, MEDAE, MSE, RMSE, MAPE, MDAPE = LSTM_mv_ModelEvaluation()
    
  def createGraph(featureCount, epochs):
    # The date from which on the date is displayed
    display_start_date = pd.Timestamp(year=2021,month=1,day=1) - timedelta(days=500)

    # Add the date column
    data_filtered_sub = data_filtered.copy()

    data_filtered_sub['Date'] = date_index

    # Add the difference between the valid and predicted prices
    train_data_len = math.ceil(np_data_scaled.shape[0] * 0.8)
    train = data_filtered_sub[:train_data_len + 1]
    valid = data_filtered_sub[train_data_len:]
        
    # Get the predicted values
    y_pred_scaled = lstm_model.predict(x_test)
    y_pred = scaler_pred.inverse_transform(y_pred_scaled)
    valid.insert(1, "Prediction", y_pred.ravel(), True)
    valid.insert(1, "Difference", valid["Prediction"] - valid["Close_Price"], True)

    valid['Date'] = pd.to_datetime(valid.Date,infer_datetime_format=True)
    train['Date'] = pd.to_datetime(train.Date,infer_datetime_format=True)

    # Zoom in to a closer timeframe
    valid = valid[valid['Date'] >= display_start_date]
    train = train[train['Date'] >= display_start_date]

    # Visualize the data
    fig, ax1 = plt.subplots(figsize=(22, 10), sharex=True)
    xt = train['Date']; yt = train[["Close_Price"]]
    xv = valid['Date']; yv = valid[["Close_Price", "Prediction"]]
    plt.title("Predictions vs Actual Values", fontsize=20)
    plt.ylabel(symbol_str, fontsize=18)
    plt.plot(xt, yt, color="#039dfc", linewidth=2.0)
    plt.plot(xv, yv["Prediction"], color="#E91D9E", linewidth=2.0)
    plt.plot(xv, yv["Close_Price"], color="black", linewidth=2.0)
    plt.legend(["Train", "Test Predictions", "Actual Values"], loc="upper left")

    # # Create the bar plot with the differences
    x = valid['Date']
    y = valid["Difference"]

    # Create custom color range for positive and negative differences
    valid.loc[y >= 0, 'diff_color'] = "#2BC97A"
    valid.loc[y < 0, 'diff_color'] = "#C92B2B"

    plt.bar(x, y, width=0.8, color=valid['diff_color'])
    plt.grid()

    fn = '_'.join(["mv", symbol_str, str(featureCount), str(epochs)+".png"])

    plt.savefig( "./models/"+fn, bbox_inches='tight')

    fig.clear()
    plt.close(fig)

    return
  
  createGraph(len(features), epochs)
  eval_list = [str(np.round(MAE,4)),
              str(np.round(MEDAE,4)),
              str(np.round(MSE,4)),
              str(np.round(RMSE,4)),
              str(np.round(MAPE,4)),
              str(np.round(MDAPE,4))]
  
  save_path           = './models'
  log_path            = os.path.join(save_path, 'log.csv')
    
  def writeModeltoFile():
  
      # Create a string of features
      features_str = '|'.join([str(c) for c in data_filtered.columns.tolist() ])
      features_count = str(np_data_scaled.shape[1])
      
      fn ="mv_" + symbol_str + "_" + str(features_count) + "_" + str(epochs) + '_lstm_model.h5'

      # Saving the model to file
      lstm_model.save(os.path.join(save_path, fn))
      

      # Update the log file re time taken to run model
      with open(log_path, "a") as text_file:
          text_file.write(fn                                           + "," +
                          symbol_str                                   + "," + 
                          datetime.now().strftime("%Y-%m-%d %H:%M:%S") + "," + 
                          ','.join(eval_list)                          + "," +
                          features_str                                 + "," +
                          features_count                               + "," +
                          str(epochs)                                  + "," +
                          str(mDur)                                    + "\n"
          )   
  
  if os.path.isfile(log_path):
      writeModeltoFile()
  else:
      with open(log_path, "a") as text_file:
          text_file.write(','.join(["Filename",
                                  "Symbol",
                                  "Date_Model_Completed",
                                  "MAE","MEDAE","MSE","RMSE","MAPE","MDAPE",
                                  "Feature_Names",
                                  "Feature_Count",
                                  "epochs",
                                  "Training_Duration_minutes\n"]))
      writeModeltoFile()
  
  return

# Create models then save the model, its graph to file, and log it in log.csv

## Search for best Hyperparameter for LSTM model

In [14]:
feature_list = [
        ["Close_Price", "sentiment"],
        ["Today_High","Today_Low","Close_Price","Volume_non_block","no_days_not_traded_since_last_traded","sentiment","is_regarding_financial_report","is_sold_pur_shares"],
        ["Today_High","Today_Low","Close_Price", "Volume_non_block"]
  ]

sStocks = inScopeStocks()
print(sStocks)

for i in [0]:

    
  # Loop, run then save models to file
  # Only three stocks are used based on the time it takes to run each model
  for symbol in sStocks[2:4]:
    s_df 	= pd.read_csv(	os.path.join('./',
                    'prices_sentiment_'+symbol+'.csv'),
                    infer_datetime_format=True, 
                    sep=','
        )
    for f in feature_list:
      for e in [25, 50]:
        create_LSTM_mv_model(
                  StockData			= s_df,
                  symbol_str		= symbol,
                  features			= f,
                  epochs				= e
        )

0         SJ
1        SGJ
2         GK
3     JMMBGL
4        JBG
5       LASD
6        CAR
7       LASM
8        SVL
9        CCC
10      LASF
11      PJAM
12        JP
13       JSE
14        KW
15     XFUND
16       RJR
17    HONBUN
18       CPJ
19       MIL
20    PROVEN
21      JAMT
22    JETCON
23       BRG
24       SEP
25       AMG
26     GENAC
27     NCBFG
28     KREMI
29      1834
30       BIL
31       KEX
32      PULS
Name: Symbol, dtype: object
1249
      Close_Price  sentiment
1244    61.032887     0.1531
1245    60.961385     0.0000
1246    61.414909     0.0000
1247    62.364313     0.0000
1248    62.677658     0.0000
[84.  0.]
(1249, 2)
1000
0.5208858677776559
1
(950, 50, 2) (950,)
(249, 50, 2) (249,)
0.3779795686719637
0.3779795686719637
100 50 2
Epoch 1/25
60/60 [==============================] - 7s 73ms/step - loss: 0.0127 - val_loss: 0.0074
Epoch 2/25
60/60 [==============================] - 4s 61ms/step - loss: 0.0050 - val_loss: 0.0075
Epoch 3/25
60/60 [===============

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:214: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:215: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


1249
      Close_Price  sentiment
1244    61.032887     0.1531
1245    60.961385     0.0000
1246    61.414909     0.0000
1247    62.364313     0.0000
1248    62.677658     0.0000
[84.  0.]
(1249, 2)
1000
0.5208858677776559
1
(950, 50, 2) (950,)
(249, 50, 2) (249,)
0.3779795686719637
0.3779795686719637
100 50 2
Epoch 1/50
60/60 [==============================] - 7s 73ms/step - loss: 0.0091 - val_loss: 0.0074
Epoch 2/50
60/60 [==============================] - 4s 63ms/step - loss: 0.0050 - val_loss: 0.0075
Epoch 3/50
60/60 [==============================] - 4s 61ms/step - loss: 0.0052 - val_loss: 0.0073
Epoch 4/50
60/60 [==============================] - 4s 61ms/step - loss: 0.0050 - val_loss: 0.0085
Epoch 5/50
60/60 [==============================] - 4s 62ms/step - loss: 0.0051 - val_loss: 0.0074
Epoch 6/50
60/60 [==============================] - 4s 62ms/step - loss: 0.0050 - val_loss: 0.0076
Epoch 7/50
60/60 [==============================] - 4s 62ms/step - loss: 0.0051 - val_loss: 0.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:214: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:215: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


1249
      Today_High  Today_Low  ...  is_regarding_financial_report  is_sold_pur_shares
1244       63.85      61.00  ...                            0.0                 0.0
1245       63.85      58.01  ...                            0.0                 0.0
1246       63.84      61.00  ...                            0.0                 0.0
1247       63.85      61.00  ...                            0.0                 0.0
1248       64.50      61.00  ...                            0.0                 0.0

[5 rows x 8 columns]
[  84.   84.   84. 2483.    0.    0.    0.    0.]
(1249, 8)
1000
0.5
1
(950, 50, 8) (950,)
(249, 50, 8) (249,)
0.5285952248750694
0.5285952248750694
400 50 8
Epoch 1/25
60/60 [==============================] - 35s 527ms/step - loss: 0.0083 - val_loss: 0.0020
Epoch 2/25
60/60 [==============================] - 31s 516ms/step - loss: 0.0039 - val_loss: 6.0628e-04
Epoch 3/25
60/60 [==============================] - 31s 516ms/step - loss: 0.0025 - val_loss: 5.4480e-04


...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:214: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:215: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


1248
      Today_High  Today_Low  Close_Price  Volume_non_block
1243       34.00      32.51    33.763762          174744.0
1244       34.00      32.00    32.707133           26573.0
1245       34.00      31.50    32.568905          652592.0
1246       33.97      31.80    33.077458           41143.0
1247       37.00      32.00    34.694071         2477532.0
[   10.29    10.2     10.26 10000.  ]
(1248, 4)
999
0.017335966644722395
1
(949, 50, 4) (949,)
(249, 50, 4) (249,)
0.02998846597462515
0.02998846597462515
200 50 4
Epoch 1/25
60/60 [==============================] - 15s 202ms/step - loss: 0.0139 - val_loss: 0.0012
Epoch 2/25
60/60 [==============================] - 11s 183ms/step - loss: 0.0010 - val_loss: 9.4038e-04
Epoch 3/25
60/60 [==============================] - 11s 185ms/step - loss: 8.3431e-04 - val_loss: 8.6966e-04
Epoch 4/25
60/60 [==============================] - 11s 186ms/step - loss: 0.0011 - val_loss: 0.0011
Epoch 5/25
60/60 [==============================] - 11s 186ms

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:214: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:215: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


1248
      Today_High  Today_Low  Close_Price  Volume_non_block
1243       34.00      32.51    33.763762          174744.0
1244       34.00      32.00    32.707133           26573.0
1245       34.00      31.50    32.568905          652592.0
1246       33.97      31.80    33.077458           41143.0
1247       37.00      32.00    34.694071         2477532.0
[   10.29    10.2     10.26 10000.  ]
(1248, 4)
999
0.017335966644722395
1
(949, 50, 4) (949,)
(249, 50, 4) (249,)
0.02998846597462515
0.02998846597462515
200 50 4
Epoch 1/50
60/60 [==============================] - 15s 199ms/step - loss: 0.0073 - val_loss: 9.5720e-04
Epoch 2/50
60/60 [==============================] - 11s 187ms/step - loss: 9.3901e-04 - val_loss: 9.5898e-04
Epoch 3/50
60/60 [==============================] - 11s 187ms/step - loss: 8.5999e-04 - val_loss: 0.0013
Epoch 4/50
60/60 [==============================] - 11s 187ms/step - loss: 8.5140e-04 - val_loss: 0.0014
Epoch 5/50
60/60 [==============================] - 1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:214: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:215: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


## Based on analysis of results above, model with epochs of 25 and the features below are the best models for us to use for our analysis

In [ ]:
feature_list = [
        ["Today_High","Today_Low","Close_Price", "Volume_non_block"],
        ["Today_High","Today_Low","Close_Price", "Volume_non_block","sentiment"]
]

sStocks = inScopeStocks()
print(sStocks)

# Loop, run then save models to file
for symbol in sStocks:
  s_df 	= pd.read_csv(	os.path.join('./',
                  'prices_sentiment_'+symbol+'.csv'),
                  infer_datetime_format=True, 
                  sep=','
      )
  for f in feature_list:
    for e in [25]:
      create_LSTM_mv_model(
                StockData			= s_df,
                symbol_str		= symbol,
                features			= f,
                epochs				= e
      )

0         SJ
1        SGJ
2         GK
3     JMMBGL
4        JBG
5       LASD
6        CAR
7       LASM
8        SVL
9        CCC
10      LASF
11      PJAM
12        JP
13       JSE
14        KW
15     XFUND
16       RJR
17    HONBUN
18       CPJ
19       MIL
20    PROVEN
21      JAMT
22    JETCON
23       BRG
24       SEP
25       AMG
26     GENAC
27     NCBFG
28     KREMI
29      1834
30       BIL
31       KEX
32      PULS
Name: Symbol, dtype: object
1256
      Close_Price  sentiment
1251    49.107108     0.0000
1252    49.438154     0.0000
1253    49.668842     0.0000
1254    50.019280     0.0772
1255    49.675486     0.0000
[20.      0.4215]
(1256, 2)
1005
0.0
1
(955, 50, 2) (955,)
(251, 50, 2) (251,)
0.33203151078176113
0.33203151078176113
100 50 2
Epoch 1/25
60/60 [==============================] - 7s 74ms/step - loss: 0.0350 - val_loss: 0.0211
Epoch 2/25
60/60 [==============================] - 4s 63ms/step - loss: 0.0246 - val_loss: 0.0211
Epoch 3/25
60/60 [====================

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:214: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:215: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


1256
      Close_Price  sentiment
1251    49.107108     0.0000
1252    49.438154     0.0000
1253    49.668842     0.0000
1254    50.019280     0.0772
1255    49.675486     0.0000
[20.      0.4215]
(1256, 2)
1005
0.0
1
(955, 50, 2) (955,)
(251, 50, 2) (251,)
0.33203151078176113
0.33203151078176113
100 50 2
Epoch 1/50
60/60 [==============================] - 7s 73ms/step - loss: 0.0305 - val_loss: 0.0218
Epoch 2/50
60/60 [==============================] - 4s 63ms/step - loss: 0.0248 - val_loss: 0.0221
Epoch 3/50
60/60 [==============================] - 4s 62ms/step - loss: 0.0259 - val_loss: 0.0210
Epoch 4/50
60/60 [==============================] - 4s 62ms/step - loss: 0.0243 - val_loss: 0.0218
Epoch 5/50
60/60 [==============================] - 4s 62ms/step - loss: 0.0250 - val_loss: 0.0273
Epoch 6/50
60/60 [==============================] - 4s 63ms/step - loss: 0.0249 - val_loss: 0.0212
Epoch 7/50
60/60 [==============================] - 4s 63ms/step - loss: 0.0246 - val_loss: 0.0211


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:214: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:215: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


1256
      Today_High  Today_Low  ...  is_regarding_financial_report  is_sold_pur_shares
1251        49.5      48.01  ...                            0.0                 0.0
1252        50.0      47.02  ...                            0.0                 0.0
1253        50.5      47.20  ...                            0.0                 0.0
1254        51.0      50.00  ...                            0.0                 1.0
1255        50.5      49.00  ...                            0.0                 0.0

[5 rows x 8 columns]
[   20.1       20.        20.     26426.         0.         0.4215
     0.         1.    ]
(1256, 8)
1005
0.0
1
(955, 50, 8) (955,)
(251, 50, 8) (251,)
0.037272727272727235
0.037272727272727235
400 50 8
Epoch 1/25
60/60 [==============================] - 35s 530ms/step - loss: 0.0077 - val_loss: 0.0027
Epoch 2/25
60/60 [==============================] - 31s 510ms/step - loss: 9.5549e-04 - val_loss: 0.0037
Epoch 3/25
60/60 [==============================] - 31s 522m

...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:214: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:215: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


1252
      Today_High  Today_Low  Close_Price  Volume_non_block
1247       45.00      42.01    44.960166         1105261.0
1248       45.80      43.90    44.999820         1149208.0
1249       44.95      43.50    44.704556           37260.0
1250       44.95      43.50    44.303998            6540.0
1251       44.25      43.50    44.117982           17576.0
[   30.02    30.      30.01 36809.  ]
(1252, 4)
1002
0.02691867124856817
1
(952, 50, 4) (952,)
(250, 50, 4) (250,)
0.09575923392612862
0.09575923392612862
200 50 4
Epoch 1/25
60/60 [==============================] - 15s 200ms/step - loss: 0.0143 - val_loss: 0.0057
Epoch 2/25
60/60 [==============================] - 11s 187ms/step - loss: 0.0018 - val_loss: 0.0049
Epoch 3/25
60/60 [==============================] - 11s 186ms/step - loss: 0.0018 - val_loss: 0.0040
Epoch 4/25
60/60 [==============================] - 11s 186ms/step - loss: 0.0017 - val_loss: 0.0039
Epoch 5/25
60/60 [==============================] - 11s 187ms/step - loss

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:214: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:215: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


1252
      Today_High  Today_Low  Close_Price  Volume_non_block
1247       45.00      42.01    44.960166         1105261.0
1248       45.80      43.90    44.999820         1149208.0
1249       44.95      43.50    44.704556           37260.0
1250       44.95      43.50    44.303998            6540.0
1251       44.25      43.50    44.117982           17576.0
[   30.02    30.      30.01 36809.  ]
(1252, 4)
1002
0.02691867124856817
1
(952, 50, 4) (952,)
(250, 50, 4) (250,)
0.09575923392612862
0.09575923392612862
200 50 4
Epoch 1/50
60/60 [==============================] - 15s 198ms/step - loss: 0.0123 - val_loss: 0.0039
Epoch 2/50
60/60 [==============================] - 11s 187ms/step - loss: 0.0018 - val_loss: 0.0042
Epoch 3/50
60/60 [==============================] - 11s 187ms/step - loss: 0.0017 - val_loss: 0.0044
Epoch 4/50
60/60 [==============================] - 11s 184ms/step - loss: 0.0016 - val_loss: 0.0040
Epoch 5/50
60/60 [==============================] - 11s 188ms/step - loss

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:214: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:215: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


# Save System Specs

In [7]:
!df -h > "./models/filesystem.txt"

In [8]:
!cat /proc/cpuinfo > "./models/cpuinfo.txt"

In [9]:
!cat /proc/meminfo > "./models/meminfo.txt"

In [10]:
cat /etc/os-release > "./models/os_description.txt"

In [11]:
!uname -a > "./models/os_bit.txt"